In [4]:
import pandas as pd
# CSV 파일에서 titleId 값을 가져오기
title_df = pd.read_csv("naver_listly_updated.csv")  # title.csv 파일을 불러오기
id_list = title_df['ID'].astype(str).tolist()  # titleId 칼럼의 값들을 문자열로 변환하여 리스트로 저장
print(len(id_list))
print(id_list[:10])

2468
['720121', '738145', '729767', '718020', '796152', '647946', '716857', '713872', '734348', '715935']


In [7]:
# 스토리 제외 크롤링 코드

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import time

id_list = id_list[:3]

# URL과 ID 리스트
base_url = 'https://comic.naver.com/webtoon/list?titleId='
urls = [f"{base_url}{num}" for num in id_list]  # titleId 값들을 사용하여 각 URL 생성

# Webdriver headless mode 설정
options = Options()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 웹 드라이버 생성
driver = webdriver.Chrome(options=options)

# 세션 설정
session = requests.Session()
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
}
retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])
session.mount('http://', HTTPAdapter(max_retries=retries))

# 데이터 저장용 리스트
data = []

# URL 리스트 순회
for url in urls:
    # Selenium으로 페이지 열기
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기

    # BeautifulSoup을 통한 HTML 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목 찾기
    title_h2 = soup.find("h2", class_="EpisodeListInfo__title--mYLjC")
    title_text = title_h2.get_text() if title_h2 else "제목을 찾을 수 없음"

    # 태그 그룹 찾기
    tags_div = soup.find("div", class_="TagGroup__tag_group--uUJza")
    if tags_div:
        tags = [tag.get_text() for tag in tags_div.find_all("a", class_="TagGroup__tag--xu0OH")]
        tags_text = ', '.join(tags)  # ','로 태그 연결
    else:
        tags_text = "태그 그룹을 찾을 수 없음"

    # 에피소드 정보 텍스트 찾기
    episode_info = soup.find("em", class_="ContentMetaInfo__info_item--utGrf")
    episode_info_text = episode_info.get_text(strip=True) if episode_info else "에피소드 정보를 찾을 수 없음"

    # 조회수 찾기
    view_count = soup.find("span", class_="EpisodeListUser__count--fNEWK")
    view_count_text = view_count.get_text(strip=True) if view_count else "조회수를 찾을 수 없음"

    # 날짜 찾기
    date_tag = soup.find("span", class_="date")
    date_text = date_tag.get_text(strip=True) if date_tag else "날짜를 찾을 수 없음"

    # 데이터를 리스트에 추가 (URL 제외)
    data.append({
        'Title': title_text,
        'Webtoon Info': episode_info_text,
        'Like Count': view_count_text,
        '1st Date': date_text,
        '#': tags_text  # '#'을 마지막에 배치
    })
    # 현재 작품
    print('현재작품 :', title_text)

    # 쉬는 시간 (10초)
    time.sleep(10)

# 드라이버 종료
driver.quit()

# 데이터 프레임 생성 및 컬럼 순서 설정
df = pd.DataFrame(data)[['Title', 'Webtoon Info', 'Like Count', '1st Date', '#']]

# 데이터 프레임 출력
df

현재작품 : 치즈인더트랩
현재작품 : 숲속의 담
현재작품 : 고래별


,Title,Webtoon Info,Like Count,1st Date,#
0,치즈인더트랩,312화 완결∙12세 이용가,"329,861",18.12.21,"#로맨스, #명작, #드라마&영화 원작웹툰, #캠퍼스로맨스, #삼각관계, #완결로맨스"
1,숲속의 담,132화 완결∙전체연령가,"331,606",19.12.23,"#판타지, #명작, #2019 지상최대공모전, #아포칼립스, #완결판타지"
2,고래별,108화 완결∙12세 이용가,"326,021",19.06.20,"#로맨스, #명작, #후회물, #성장드라마, #완결로맨스"


In [12]:
df.to_csv('webtoon_plus_info.csv', index=False, encoding='utf-8-sig')

In [12]:
# 스토리 크롤링 코드

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

id_list = id_list[:3]
# titleId 칼럼의 값들을 문자열로 변환하여 리스트로 저장
#id_list = ['796152', '817346']

# URL과 ID 리스트
base_url = 'https://comic.naver.com/webtoon/list?titleId='
urls = [f"{base_url}{num}" for num in id_list]  # titleId 값들을 사용하여 각 URL 생성

# Webdriver headless mode 설정
options = Options()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")

# 웹 드라이버 생성
driver = webdriver.Chrome(options=options)

# 데이터 저장용 리스트
data = []

# URL 리스트 순회
for url in urls:
    # Selenium으로 페이지 열기
    driver.get(url)
    time.sleep(2)  # 페이지 로딩 대기
    
    # BeautifulSoup을 통한 HTML 파싱
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목 찾기
    title_h2 = soup.find("h2", class_="EpisodeListInfo__title--mYLjC")
    title_text = title_h2.get_text() if title_h2 else "제목을 찾을 수 없음"
    
    # <p> 태그의 텍스트 크롤링
    story_p = soup.find("p", class_="EpisodeListInfo__summary--Jd1WG")
    story_text = story_p.get_text(strip=True) if story_p else "스토리를 찾을 수 없음"
    
    # 데이터를 리스트에 추가
    data.append({'Title': title_text,
                 'Story': story_text})
    print('현재작품 :', title_text)
    
    # 쉬는 시간
    time.sleep(2)

# 드라이버 종료
driver.quit()

# 데이터 프레임 생성 및 컬럼 순서 설정
df = pd.DataFrame(data)[['Title', 'Story']]


# 결과 출력
df

현재작품 : 치즈인더트랩
현재작품 : 숲속의 담
현재작품 : 고래별


,Title,Story
0,치즈인더트랩,"평범한 여대생 홍설, 그리고 어딘가 수상한 선배 유정.\n미묘한 관계의 이들이 펼쳐..."
1,숲속의 담,어느 순간부터 성장이 멈춘 '담'은\n사람들을 피해 숲속으로 떠난다.
2,고래별,1926년 일제 식민 지배 하의 조선. \n17세 소녀 수아는 군산 일대 친일파 대...


In [12]:
# 전체 합친 코드

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time

# ID 리스트 (예: ['796152', '817346', '703846'] 등)
id_list = id_list[:3]  # 필요 시 직접 지정 가능

# URL 생성
base_url = 'https://comic.naver.com/webtoon/list?titleId='
urls = [f"{base_url}{num}" for num in id_list]

# WebDriver 설정
options = Options()
options.add_argument('headless')
options.add_argument('window-size=1920x1080')
options.add_argument("disable-gpu")
driver = webdriver.Chrome(options=options)

# 데이터 저장 리스트
data = []

# URL 순회하며 정보 수집
for url in urls:
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 제목
    title_h2 = soup.find("h2", class_="EpisodeListInfo__title--mYLjC")
    title_text = title_h2.get_text() if title_h2 else "제목을 찾을 수 없음"

    # 줄거리(스토리)
    story_p = soup.find("p", class_="EpisodeListInfo__summary--Jd1WG")
    story_text = story_p.get_text(strip=True) if story_p else "스토리를 찾을 수 없음"

    # 태그
    tags_div = soup.find("div", class_="TagGroup__tag_group--uUJza")
    if tags_div:
        tags = [tag.get_text() for tag in tags_div.find_all("a", class_="TagGroup__tag--xu0OH")]
        tags_text = ', '.join(tags)
    else:
        tags_text = "태그 그룹을 찾을 수 없음"

    # 에피소드 수 정보
    episode_info = soup.find("em", class_="ContentMetaInfo__info_item--utGrf")
    episode_info_text = episode_info.get_text(strip=True) if episode_info else "에피소드 정보를 찾을 수 없음"

    # 조회수 (좋아요 수)
    view_count = soup.find("span", class_="EpisodeListUser__count--fNEWK")
    view_count_text = view_count.get_text(strip=True) if view_count else "조회수를 찾을 수 없음"

    # 첫 공개일
    date_tag = soup.find("span", class_="date")
    date_text = date_tag.get_text(strip=True) if date_tag else "날짜를 찾을 수 없음"

    # 저장
    data.append({
        'Title': title_text,
        'Story': story_text,
        'Webtoon Info': episode_info_text,
        'Like Count': view_count_text,
        '1st Date': date_text,
        '#': tags_text
    })

    print("현재작품 :", title_text)
    time.sleep(10)

# 드라이버 종료
driver.quit()

# DataFrame 생성 및 출력
df = pd.DataFrame(data)[['Title', 'Story', 'Webtoon Info', 'Like Count', '1st Date', '#']]
df

현재작품 : 치즈인더트랩
현재작품 : 숲속의 담
현재작품 : 고래별


,Title,Story,Webtoon Info,Like Count,1st Date,#
0,치즈인더트랩,"평범한 여대생 홍설, 그리고 어딘가 수상한 선배 유정.\n미묘한 관계의 이들이 펼쳐...",312화 완결∙12세 이용가,"329,897",18.12.21,"#로맨스, #명작, #드라마&영화 원작웹툰, #캠퍼스로맨스, #삼각관계, #완결로맨스"
1,숲속의 담,어느 순간부터 성장이 멈춘 '담'은\n사람들을 피해 숲속으로 떠난다.,132화 완결∙전체연령가,"331,642",19.12.23,"#판타지, #명작, #2019 지상최대공모전, #아포칼립스, #완결판타지"
2,고래별,1926년 일제 식민 지배 하의 조선. \n17세 소녀 수아는 군산 일대 친일파 대...,108화 완결∙12세 이용가,"326,022",19.06.20,"#로맨스, #명작, #후회물, #성장드라마, #완결로맨스"


In [ ]:
df.to_csv('1-99.csv', index=False, encoding='utf-8-sig')